# SimPy Example for IE 306.02
This example shows how a simple G/G/1 system (1 server with an infinite capacity queue, random interarrival and service times) can be modeled with a process-interaction view using the SimPy library. 

In this example customers place calls to the call center of a company at randomn times. There is only a single operator in this call center, and picks up the first call waiting when she is available. The customers are assumed to be extremely paint, as they wait as long as it gets to talk to the operator.

In [1]:
import simpy
import random

Define a set of globals that define the characteristics of the model instance to be simulated. This includes the seed (RANDOM_SEED) for the random number generators, and key parameters for the interarrival (i.e. mean arrival rate) and service time (i.e. lower and upper bounds for the range) distribution.

In [2]:
RANDOM_SEED = 978
INTERARRIVAL_RATE = 0.1 
SERVICE_RANGE = [50, 90]
random.seed(RANDOM_SEED)

Define the necessary set of arrays for bookkeeping

In [3]:
service_times = [] #Duration of the conversation between the customer and the operator (Service time)
queue_w_times = [] #Time spent by a customer while it waits for the operator (Queue waiting time Wq)

* The class definition for the customers arriving at the modeled system. When they are created, they immediatelly initiate a call (i.e. activate the call process). 

* Once a call is initiated, this is registered as a request to the operator resource. The customer is put on hold until the resource activates it back. 

* When the resource is available, the customer is activated and it then initiates the ask_question process. The duration of a question-answer session is determined randomly according to a uniform distribution.

In [129]:
class Customer(object):
    def __init__(self, name, env, opr):
        self.env = env
        self.name = name
        self.arrival_t = self.env.now
        self.action = env.process(self.call())
    
    
    def call(self):
        print('%s initiated a call at %g' % (self.name, self.env.now))
 
        with operator.request() as req:
            yield req
            print('%s is assigned to an operator at %g' % (self.name, self.env.now))
            queue_w_times.append(self.env.now - self.arrival_t)
            yield self.env.process(self.ask_question())
            print('%s is done at %g' % (self.name, self.env.now))
            
    def ask_question(self):
        duration = random.uniform(*SERVICE_RANGE)
        yield self.env.timeout(duration)
        service_times.append(duration)
        
        

In [130]:
def customer_generator(env, operator):
    """Generate new cars that arrive at the gas station."""
    for i in range(10):
        yield env.timeout(random.expovariate(INTERARRIVAL_RATE))
        customer = Customer('Cust %s' %(i+1), env, operator)  

In [131]:
env = simpy.Environment()
operator = simpy.Resource(env, capacity = 1)
env.process(customer_generator(env, operator))
env.run() 

Cust 1 initiated a call at 44.1894
Cust 1 is assigned to an operator at 44.1894
Cust 2 initiated a call at 45.8734
Cust 3 initiated a call at 62.0814
Cust 4 initiated a call at 69.7013
Cust 5 initiated a call at 75.4224
Cust 6 initiated a call at 91.9224
Cust 1 is done at 97.9088
Cust 2 is assigned to an operator at 97.9088
Cust 7 initiated a call at 109.484
Cust 8 initiated a call at 115.873
Cust 9 initiated a call at 116.609
Cust 10 initiated a call at 136.475
Cust 2 is done at 172.276
Cust 3 is assigned to an operator at 172.276
Cust 3 is done at 229.78
Cust 4 is assigned to an operator at 229.78
Cust 4 is done at 315.98
Cust 5 is assigned to an operator at 315.98
Cust 5 is done at 376.526
Cust 6 is assigned to an operator at 376.526
Cust 6 is done at 441.072
Cust 7 is assigned to an operator at 441.072
Cust 7 is done at 523.873
Cust 8 is assigned to an operator at 523.873
Cust 8 is done at 603.025
Cust 9 is assigned to an operator at 603.025
Cust 9 is done at 691.986
Cust 10 is ass

In [125]:
print queue_w_times

[0.0, 52.035354931146024, 110.1948451847324, 160.07892668870812, 240.55796255507212, 284.6039007134052, 331.58790736643346, 408.0003801503316, 486.4154249382668, 555.5113070098521, 0.0, 46.06435021197851, 100.81374880663847, 165.43206513461266, 215.89896415211186, 274.73277374614304, 323.67864703565846, 399.13748225202175, 455.94973693899067, 525.5860737687431]


In [123]:
print service_times

[53.719422936944206, 74.36745758097598, 57.50398356266595, 86.20015959138377, 60.54594847560017, 64.54573727938265, 82.80127819535485, 79.15142152985119, 88.96147734612259, 80.14527961665469]
